In [1]:
import pandas as pd
import numpy as np

In [2]:
import ftplib

In [3]:
ftp = ftplib.FTP(host='f25-preview.runhosting.com', user='3185129_EOI', passwd='FTP_EOI1920')

In [4]:
# Vista de los archivos del directorio
ftp.dir()

drwxr-xr-x   2 3185129_EOI 1111           58 Nov 24  2019 .
drwxr-xr-x   2 3185129_EOI 1111           58 Nov 24  2019 ..
-rw-r--r--   1 3185129_EOI 1111        33182 Nov 24  2019 books.xml
-rw-r--r--   1 3185129_EOI 1111         3841 Oct 20  2019 colors.json
-rw-r--r--   1 3185129_EOI 1111       763509 Oct 20  2019 OptaF24.xml


In [5]:
filename = ('OptaF24.xml', 'r')
ftp.retrbinary('RETR ' + 'OptaF24.xml', open('OptaF24.xml', 'wb').write)

'226 Transfer complete'

In [6]:
ftp.quit()

'221 Goodbye.'

In [7]:
import xml.etree.ElementTree as ET

In [9]:
tree = ET.parse('OptaF24.xml')
raiz = tree.getroot()
game = tree.findall('Game')

In [12]:
raiz.tag

'Games'

In [14]:
events = game[0].findall('Event')
events

[<Element 'Event' at 0x0000022F01E04C28>,
 <Element 'Event' at 0x0000022F024BA228>,
 <Element 'Event' at 0x0000022F024BA7C8>,
 <Element 'Event' at 0x0000022F024BA9A8>,
 <Element 'Event' at 0x0000022F024BAB88>,
 <Element 'Event' at 0x0000022F024BE048>,
 <Element 'Event' at 0x0000022F024BE4A8>,
 <Element 'Event' at 0x0000022F024BE728>,
 <Element 'Event' at 0x0000022F024BE908>,
 <Element 'Event' at 0x0000022F024BEAE8>,
 <Element 'Event' at 0x0000022F024BECC8>,
 <Element 'Event' at 0x0000022F024C2228>,
 <Element 'Event' at 0x0000022F024C2728>,
 <Element 'Event' at 0x0000022F024C29A8>,
 <Element 'Event' at 0x0000022F024C2C28>,
 <Element 'Event' at 0x0000022F024C6188>,
 <Element 'Event' at 0x0000022F024C65E8>,
 <Element 'Event' at 0x0000022F024C6A48>,
 <Element 'Event' at 0x0000022F024C6F48>,
 <Element 'Event' at 0x0000022F024CA228>,
 <Element 'Event' at 0x0000022F024CA408>,
 <Element 'Event' at 0x0000022F024CA728>,
 <Element 'Event' at 0x0000022F024CA9A8>,
 <Element 'Event' at 0x0000022F024

In [19]:
equipos = {raiz[0].attrib['home_team_id']: raiz[0].attrib['home_team_name'],
           raiz[0].attrib['away_team_id']: raiz[0].attrib['away_team_name']}
equipos

{'30': 'Bolton Wanderers', '43': 'Manchester City'}

In [21]:
len(events)

1673

In [32]:
x_origin = []
y_origin = []
x_destination = []
y_destination = []
outcome = []
minute = []
second = []
half = []
team = []

for event in events:
    if event.attrib.get("type_id") == "1":
        outcome.append(event.attrib.get("outcome"))
        half.append(event.attrib.get("period_id"))
        minute.append(event.attrib.get("min"))
        second.append(event.attrib.get('sec'))
        x_origin.append(event.attrib.get('x'))
        y_origin.append(event.attrib.get('y'))
        team.append(equipos[event.attrib.get('team_id')])
        for q in event:
            if q.attrib.get('qualifier_id') == '140':
                x_destination.append(q.attrib.get('value'))
            if q.attrib.get('qualifier_id') == '141':
                y_destination.append(q.attrib.get('value'))

In [34]:
df = pd.DataFrame({"team":team, 
                   "half":half,
                   "minute":minute,
                   "second":second,
                   "x_origin":x_origin,
                   "y_origin":y_origin,
                   "x_destination":x_destination,
                   "y_destination":y_destination,
                   "outcome":outcome})

df

,team,half,minute,second,x_origin,y_origin,x_destination,y_destination,outcome
0,Manchester City,1,0,1,50.1,50.0,52.4,49.1,1
1,Manchester City,1,0,2,48.2,49.1,29.0,76.5,1
2,Manchester City,1,0,19,27.8,100.0,49.3,93.4,0
3,Bolton Wanderers,1,0,23,50.9,20.0,63.6,30.1,1
4,Manchester City,1,0,30,29.3,73.0,30.2,55.6,1
...,...,...,...,...,...,...,...,...,...
941,Bolton Wanderers,2,93,42,16.3,82.5,71.7,70.7,0
942,Manchester City,2,93,45,28.6,27.6,34.2,9.5,1
943,Manchester City,2,93,46,34.4,9.1,63.0,20.7,1
944,Manchester City,2,93,47,62.4,20.7,61.8,12.5,1


In [35]:
df_pases = df.loc[df.outcome=="1"]
df_pases

,team,half,minute,second,x_origin,y_origin,x_destination,y_destination,outcome
0,Manchester City,1,0,1,50.1,50.0,52.4,49.1,1
1,Manchester City,1,0,2,48.2,49.1,29.0,76.5,1
3,Bolton Wanderers,1,0,23,50.9,20.0,63.6,30.1,1
4,Manchester City,1,0,30,29.3,73.0,30.2,55.6,1
5,Manchester City,1,0,32,31.9,53.6,42.3,34.9,1
...,...,...,...,...,...,...,...,...,...
940,Manchester City,2,93,32,70.1,2.8,86.5,2.7,1
942,Manchester City,2,93,45,28.6,27.6,34.2,9.5,1
943,Manchester City,2,93,46,34.4,9.1,63.0,20.7,1
944,Manchester City,2,93,47,62.4,20.7,61.8,12.5,1


In [38]:
df_pases['distancia'] = df_pases['x_destination'].astype('float') - df_pases['x_origin'].astype('float')

C:\Users\rzambrano\Anaconda3\envs\general\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
df2 = df_pases.loc[df_pases.distancia>=20]
df2.groupby('team')['distancia'].count()

team
Bolton Wanderers    35
Manchester City     32
Name: distancia, dtype: int64